<a href="https://colab.research.google.com/github/whkwls2653/Emotion-Recognition/blob/main/tip_colab_%ED%8C%8C%EC%9D%BC%EC%A0%80%EC%9E%A5%EB%B0%8F%EC%97%85%EB%A1%9C%EB%93%9C_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
!python --version

Python 3.9.16


In [3]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-fahkfex6
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-fahkfex6
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [5]:
!pip uninstall numpy==1.24.2
!pip install numpy==1.23.1



Found existing installation: numpy 1.23.1
Uninstalling numpy-1.23.1:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.9
    /usr/local/lib/python3.9/dist-packages/numpy-1.23.1.dist-info/*
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libopenblas64_p-r0-742d56dc.3.20.so
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.9/dist-packages/numpy/*
Proceed (Y/n)? n
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


NameError: ignored

In [8]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import gluonnlp as nlp

from tqdm import tqdm, tqdm_notebook

In [9]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [10]:
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [15]:
import pandas as pd
sess1_f = pd.read_csv('/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/annotation/Session01_F_res.csv')

In [16]:
sess1_f.sample(n=10)


,Numb,Wav,Unnamed: 2,ECG,Unnamed: 4,E4-EDA,Unnamed: 6,E4-TEMP,Unnamed: 8,Segment ID,...,Unnamed: 33,Eval08F,Unnamed: 35,Unnamed: 36,Eval09M,Unnamed: 38,Unnamed: 39,Eval10F,Unnamed: 41,Unnamed: 42
387,387.0,100.182,102.306,0.1771373,0.02892037,0.112983,0.127073,29.37,29.37,Sess01_impro03_M009,...,3,angry,2,2,neutral,2,3,neutral,2,3
291,291.0,7.631,14.191,-0.0469956,-0.003615046,0.530615,0.549828,29.67,29.67,Sess01_impro01_M001,...,3,neutral,2,3,neutral,4,2,neutral,3,3
313,313.0,261.833,281.84,0.701319,0.1807523,1.36456,1.560732,29.85,29.73,Sess01_impro01_M012,...,3,neutral,2,2,angry,2,4,neutral,2,3
187,187.0,232.495,238.139,-0.2819736,-0.1626771,0.1577,0.1513,29.75,29.77,Sess01_script04_F018,...,4,angry,1,4,angry,2,4,angry,1,4
29,29.0,209.768,216.907,-0.1482169,-0.469956,0.504298,0.577521,30.81,30.81,Sess01_script01_F013,...,3,fear,1,4,fear,2,4,neutral,1,4
119,119.0,400.644,410.95,-0.0542257,-0.3036639,3.285265,2.331881,30.49,30.45,Sess01_script02_M032,...,4,happy,5,4,neutral,4,3,happy,4,3
177,177.0,146.688,169.635,0.003615046,-0.02892037,0.188418,0.184579,29.69,29.69,Sess01_script04_M014,...,2,neutral,2,2,neutral,2,3,neutral,1,3
379,379.0,45.063,51.559,-0.1915975,0.003615046,0.125792,0.150128,29.21,29.23,Sess01_impro03_M006,...,3,neutral,2,2,neutral,2,3,neutral,2,3
112,112.0,345.929,356.544,-0.007230093,0.007230093,1.788715,1.862952,30.53,30.53,Sess01_script02_F030,...,3,happy,4,3,neutral,4,3,neutral,4,3
93,93.0,210.341,213.986,-0.2060577,0,0.461046,0.467446,30.75,30.75,Sess01_script02_F020,...,3,surprise,3,3,neutral,3,3,neutral,3,3


In [23]:
segments=sess1_f['Segment ID']

In [26]:
!echo "# Emotion-Recognition" >> README.md
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/whkwls2653/Emotion-Recognition.git
!git push -u origin main

Initialized empty Git repository in /content/.git/

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@2af764852eed.(none)')
error: refname refs/heads/master not found
fatal: Branch rename failed
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/whkwls2653/Emotion-Recognition.git'


In [24]:
texts=[]
for i in range(len(segments)):
  filename='/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/annotation/Session01_F_res.csv'
  open(filename, mode='r')
  texts[i]=segments[i]


IndentationError: ignored

In [ ]:
!ls /gdrive/My\ Drive/Colab\ Notebooks/data

In [ ]:
accidents = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data/accidents.txt', delimiter='\t')
accidents

메뉴창 이용하기
--
- 좌측의 메뉴창 '파일'에서 마운트 되어 있는 구글 드라이브의 내용을 확인할 수 있다.
- PC에서 직접 업로드/다운로드 할 수 있다.

## 구글 드라이브에 파일을 보내는 방법

In [ ]:
!ls /gdrive/My\ Drive/Colab\ Notebooks/data

In [ ]:
accidents.to_csv('/gdrive/My Drive/Colab Notebooks/data/new_file_accidents.txt')
!ls /gdrive/My\ Drive/Colab\ Notebooks/data

아래는 PyDrive, 구글 시트, 구글 클라우드를 이용하는 방법
==

## PyDrive

The example below shows 1) authentication, 2) file upload, and 3) file download. More examples are available in the [PyDrive documentation](https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html)

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html

# 2. Create & upload a file text file.
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
with open('/gdrive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /gdrive/foo.txt

## Drive REST API

The first step is to authenticate.

In [ ]:
from google.colab import auth
auth.authenticate_user()

Now we can construct a Drive API client.

In [ ]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

With the client created, we can use any of the functions in the [Google Drive API reference](https://developers.google.com/drive/v3/reference/). Examples follow.


## Creating a new Drive file with data from Python

In [ ]:
# Create a local file to upload.
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

In [ ]:
# Upload the file to Drive. See:
#
# https://developers.google.com/drive/v3/reference/files/create
# https://developers.google.com/drive/v3/web/manage-uploads
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

After executing the cell above, a new file named 'Sample file' will appear in your [drive.google.com](https://drive.google.com/) file list. Your file ID will differ since you will have created a new, distinct file from the example above.

## Downloading data from a Drive file into Python

In [ ]:
# Download the file we just uploaded.
#
# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1uBtlaggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'target_file_id'

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

## Google Sheets

Our examples below will use the existing open-source [gspread](https://github.com/burnash/gspread) library for interacting with Sheets.

First, we'll install the package using `pip`.

In [ ]:
!pip install --upgrade -q gspread

Next, we'll import the library, authenticate, and create the interface to sheets.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Below is a small set of gspread examples. Additional examples are shown on the [gspread Github page](https://github.com/burnash/gspread#more-examples).

## Creating a new sheet with data from Python

In [ ]:
sh = gc.create('A new spreadsheet')

After executing the cell above, a new spreadsheet will be shown in your sheets list on [sheets.google.com](http://sheets.google.com/).

In [ ]:
# Open our new sheet and add some data.
worksheet = gc.open('A new spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

After executing the cell above, the sheet will be populated with random numbers in the assigned range.

## Downloading data from a sheet into Python as a Pandas DataFrame

We'll read back to the data that we inserted above and convert the result into a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

(The data you observe will differ since the contents of each cell is a random number.)

In [ ]:
# Open our new sheet and read some data.
worksheet = gc.open('A new spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

# Google Cloud Storage (GCS)

We'll start by authenticating to GCS and creating the service client.

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Upload a file from Python to a GCS bucket

We'll start by creating the sample file to be uploaded.

In [ ]:
# Create a local file to upload.
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

Next, we'll upload the file using the `gsutil` command, which is included by default on Colab backends.

In [ ]:
# First, we need to set our project. Replace the assignment below
# with your project ID.
project_id = 'Your_project_ID_here'

In [ ]:
!gcloud config set project {project_id}

In [ ]:
import uuid

# Make a unique bucket to which we'll upload the file.
# (GCS buckets are part of a single global namespace.)
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/mb
!gsutil mb gs://{bucket_name}

In [ ]:
# Copy the file to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

In [ ]:
# Finally, dump the contents of our newly copied file to make sure everything worked.
!gsutil cat gs://{bucket_name}/to_upload.txt

### Using Python

This section demonstrates how to upload files using the native Python API rather than `gsutil`.

This snippet is based on [a larger example](https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py) with additional uses of the API.

In [ ]:
# The first step is to create a bucket in your cloud project.
#
# Replace the assignment below with your cloud project ID.
#
# For details on cloud projects, see:
# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'Your_project_ID_here'

In [ ]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

# Generate a random bucket name to which we'll upload the file.
import uuid
bucket_name = 'colab-sample-bucket' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

The cell below uploads the file to our newly created bucket.

In [ ]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name, 
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Once the upload has finished, the data will appear in the cloud console storage browser for your project:

https://console.cloud.google.com/storage/browser?project=YOUR_PROJECT_ID_HERE

## Downloading a file from GCS to Python

Next, we'll download the file we just uploaded in the example above. It's as simple as reversing the order in the `gsutil cp` command.

In [ ]:
# Download the file.
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt
  
# Print the result to make sure the transfer worked.
!cat /tmp/gsutil_download.txt

### Using Python

We repeat the download example above using the native Python API.

In [ ]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

In [ ]:
# Inspect the file we downloaded to /tmp
!cat /tmp/downloaded_from_gcs.txt